In [1]:
import pandas
import warnings
import plotly.offline as py
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')
py.init_notebook_mode(connected=True)

try:
    file_path = os.path.join('../sample-data/dataset.csv')

    dataset = pandas.read_csv(file_path)
    
except FileNotFoundError:
    dataset = None
    print(f"Error: The file at {file_path} was not found.")

except pandas.errors.EmptyDataError:
    dataset = None
    print("Error: The file is empty.")
    
except pandas.errors.ParserError:
    dataset = None
    print("Error: The file could not be parsed.")

feature_cols = ["brand", "model", "year", "transmission", "mileage", "fuelType", "mpg", "engineSize"]

target_col = ["price"]

categorical_cols = ["brand", "model", "transmission", "fuelType"]

if dataset is not None:
    desired_columns = ["brand", "model", "year", "price", "transmission", "mileage", "fuelType", "mpg", "engineSize"]

    dataset = dataset[desired_columns]

try:
    label_encoder = LabelEncoder()
    dataset['brand_encoded'] = label_encoder.fit_transform(dataset['brand'])
    dataset['model_encoded'] = label_encoder.fit_transform(dataset['model'])
    dataset['transmission_encoded'] = label_encoder.fit_transform(dataset['transmission'])
    dataset['fuelType_encoded'] = label_encoder.fit_transform(dataset['fuelType'])

    X = dataset[['brand_encoded', 'model_encoded', 'year', 'transmission_encoded', 'mileage', 'fuelType_encoded', 'mpg', 'engineSize' ]]
    Y = dataset[target_col]

except KeyError as e:
    X, Y = None
    print(f"Error: The specified column {e} does not exist in the dataframe.")
    
except Exception as e:
    X, Y = None
    print(f"An unexpected error occurred during preprocessing: {e}")

x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=(0.7), random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train)
y_val = y_scaler.transform(y_val)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_mlp = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=32, callbacks=[early_stopping])

model.save('../trained-data/multilayer-perceptron-model-training.h5') 

print("success")

ModuleNotFoundError: No module named 'tensorflow'